In [19]:
import os
import time
from dotenv import load_dotenv
from pinecone import Pinecone
from IPython.display import display, Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from pinecone import ServerlessSpec



In [20]:
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key)
client = OpenAI()

In [21]:
index_name = "ia-sensor"

if not pc.has_index(index_name):
   pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

# Sem prompt

In [22]:
# Upload do PDFs
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

#  Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Consulta para buscar informações
pergunta = "Como faço para congifurar a central para o cliente?"

pergunta_com_contexto = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}"

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content)) 



Para configurar a central para o cliente, siga os seguintes passos:

1. Acesse o PUTTY e execute o comando sensorgw appConfig.
2. Confirme a configuração padrão do servidor.
3. Insira o nome do cliente no campo Client WebApp.
4. Insira o login e senha do cliente.
5. Verifique a conexão e o modem.
6. Defina a operadora no campo Carrier.
7. Sincronize as configurações com o comando sync.
8. Pare o serviço com sensorgw stop.
9. Configure o HAMACHI conforme necessário.
10. Acesse o portal logmein para aceitar solicitações pendentes.
11. Verifique se os sensores estão enviando dados para o portal do cliente.
12. Realize testes de conexão e envio de dados.
13. Acesse o portal do cliente para configurar as redes.
14. Salve e aplique um Hard Reboot na central para gravar as informações.
15. Separe o kit com a central e a fonte testada.
16. Preencha a planilha de Clientes com os dados da central e sensores.

Esses passos devem ajudar a configurar a central para o cliente de forma adequada.

# Com prompt

In [ ]:
# Upload do PDFs
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)

# Dividir texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)

# Gera embeddings
embeddings = OpenAIEmbeddings()

# Criação do armazenamento vetorial no Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=split_texts, 
    embedding=embeddings, 
    index_name=index_name 
)

# Configuração do modelo
llm = ChatOpenAI(   
    model='gpt-3.5-turbo',  
    temperature=0.0  
)  

# Configuração de mecanismo de perguntas e respostas
retriever = vectorstore.as_retriever()

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)

contexto = "\n".join(texts) 

# Prompt aprimorado para respostas detalhadas e passo a passo
prompt = f"""
Você é um assistente especializado em responder perguntas com base exclusivamente nas informações fornecidas no contexto. 
Sua resposta deve ser extremamente detalhada e organizada em um formato passo a passo. 
Se houver etapas a serem seguidas, descreva cada uma delas claramente. 
Se a resposta não puder ser encontrada no contexto, diga claramente que não sabe, sem adicionar informações externas ou especulações.

Baseado no seguinte contexto:
{contexto}

Responda à pergunta:
{pergunta}

Formato esperado:
1. Passo 1: [descrição detalhada]
2. Passo 2: [descrição detalhada]
...

Se a resposta não estiver no contexto acima, responda: 'Não encontrei essa informação no material disponível.'
"""

# Consulta para buscar informações
pergunta = "Como faço para configurar ao central para o cliente em campo?"

pergunta_com_contexto = prompt

resposta = llm(pergunta_com_contexto)

display(Markdown(resposta.content))

